# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1632541766607_0037'

In [3]:
pd.options.display.max_columns = 35

In [4]:
spark.stop()

spark = SparkSession.builder \
                    .config('spark.dynamicAllocation.maxExecutors', 50) \
                    .config('spark.executor.cores', '3') \
                    .config('spark.executor.memory', '15g') \
                    .config('spark.driver.memory', '40g') \
                    .config('spark.python.worker.memory', '2g') \
                    .config('spark.sql.shuffle.partitions', '2000') \
                    .config('spark.network.timeout', '2000') \
                    .config('spark.sql.broadcastTimeout', '2000') \
                    .config('spark.driver.maxResultSize', '6g') \
                    .config('mapreduce.fileoutputcommitter.algorithm.version', '2') \
                    .getOrCreate()
            
spark.sparkContext.applicationId

'application_1632541766607_0038'

In [5]:
from foresight.discern import broadcast_discern, push_discern, pop_discern

### care management new default will be 5e259fd5-75b5-4d49-82d3-2d4e92dca831 for any new clients or 
###  any algorithm wiki not line for line with Cerner Standard then would be using the new context  

pd.set_option('display.max_colwidth', None)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'   ##<------------- change it to your stack without the v1/

contextid = ['da0acee6-1e2e-4384-9fc7-a58c259d0c50', '5E259FD575B54D4982D32D4E92DCA831',
             '6e70bfd2-f178-48bb-b19a-31ecdfb49ee7'] 

# Loop that removes all hypthens and captializes all letters. Sets up the file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    print(contextid)

defaultcontext = 'DA0ACEE61E2E43849FC7A58C259D0C50'   
defaultcontext1 = '5E259FD575B54D4982D32D4E92DCA831' 

DA0ACEE61E2E43849FC7A58C259D0C50
5E259FD575B54D4982D32D4E92DCA831
6E70BFD2F17848BBB19A31ECDFB49EE7


In [6]:
db = "real_world_data_2021_Q2"
spark.sql("USE {}".format(db))
print("Using database: {},".format(db))

Using database: real_world_data_2021_Q2,


In [7]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,real_world_data_2021_q2,allergy,False
1,real_world_data_2021_q2,clinical_event,False
2,real_world_data_2021_q2,condition,False
3,real_world_data_2021_q2,demographics,False
4,real_world_data_2021_q2,encounter,False
5,real_world_data_2021_q2,immunization,False
6,real_world_data_2021_q2,lab,False
7,real_world_data_2021_q2,measurement,False
8,real_world_data_2021_q2,medication,False
9,real_world_data_2021_q2,medication_administration,False


### Create Variables for Queries

In [8]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [9]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS {userName}""".format(**{
    'userName':userName
})).show()

++
||
++
++



In [10]:
spark.sql(f"""select * FROM {userName}.{algorithmName}_Hospital h  """).printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- tenant_bed_size: string (nullable = true)
 |-- tenant_speciality: string (nullable = true)
 |-- tenant_segment: string (nullable = true)
 |-- tenant_zip_code: string (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- dischargeDate: string (nullable = true)
 |-- length_of_stay: integer (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- deceased: integer (nullable = true)
 |-- discharged_to_skilled_nursing_facility: integer (nullable = true)
 |-- discharged_to_hospice: integer (nullable = true)
 |-- died_in_the_hospital: integer (nullable = true)
 |-- discharged_to_rehab_facility: integer (nullable = true)
 |-- discharged_to_long_term_care_facility: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- gender: string (nullable = true)



In [11]:
spark.sql("""select * from lab """).printSchema()

root
 |-- labid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- labcode: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- loincclass: string (nullable = true)
 |-- type: string (nullable = true)
 |-- servicedate: string (nullable = true)
 |-- serviceperiod: struct (nullable = true)
 |    |-- startDate: string (nullable = true)
 |    |-- endDate: string (nullable = true)
 |-- typedvalue: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- textValue: 

In [12]:
spark.sql(f"""


    SELECT distinct 
        h.tenant
        ,count(distinct h.personId) as patients
        
            FROM {userName}.{algorithmName}_Hospital h  
            
  group by 1          

  order by 1
    


""").toPandas()

,tenant,patients
0,1,1862
1,2,30
2,3,1340
3,4,18
4,5,130
...,...,...
91,98,10
92,99,255
93,100,205
94,101,471


In [13]:
spark.sql(f"""


    SELECT distinct 
        count(distinct h.personId) as patients
        
            FROM {userName}.{algorithmName}_Hospital h  
            
 

""").toPandas()

,patients
0,177699


In [14]:
spark.sql(f"""
select distinct l.type
        
     FROM {db}.lab l
         WHERE NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                
""").toPandas()

,type
0,TEXT
1,DATE
2,NUMERIC
3,CODIFIED


In [30]:
labResultsDf = spark.sql(f"""
   select h.tenant 
         ,h.personId
         ,h.encounterId
         ,l.labid
         ,l.serviceDate
         ,l.labcode as labcode
         ,l.labcode.standard.id as labcode_id
         ,l.labcode.standard.primaryDisplay as labcode_display
         ,l.labcode.standard.codingSystemId as labcode_system_id
         ,l.type
         ,case  
           when l.type = 'NUMERIC' then cast(typedvalue.numericValue.value as string)
           when l.type = 'TEXT' then cast(typedvalue.textvalue.value as string)
           when l.type = 'DATE' then cast(typedvalue.datevalue.date as string)
           else cast(typedvalue.codifiedValues.values.value.standard.primaryDisplay as string)  
          end labcode_value   
         ,l.unitofmeasure.standard.id as labcode_unitofmeasure_id
         ,l.unitofmeasure.standard.primaryDisplay as labcode_unitofmeasure_display
         ,l.unitofmeasure.standard.codingSystemId as labcode_unitofmeasure_system_id 
         ,l.interpretation.standard.id as labcode_interpretation_id
         ,l.interpretation.standard.primaryDisplay as labcode_interpretation_display
         ,l.interpretation.standard.codingSystemId as labcode_interpretation_system_id 
   

  
        FROM {userName}.{algorithmName}_Hospital h  
        

       

        JOIN {db}.lab l
            ON  h.encounterId = l.encounterId
                and h.personId = l.personID
                and h.tenant = l.tenant
                
                 AND HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, 
                    array('COVID_19_EVALUATION_NOTE',
'COVID_19_INITIAL_EVALUATION_FORM',
'EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM',
'INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION',
'INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY',
'INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION',
'INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY',
'MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA',
'POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS',
'RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE',
'SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION',
'SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION',
'SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA',
'SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY',
'SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY',
'SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT',
'SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE',
'SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING',
'SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT',
'SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
'SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1',
'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2',
'SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION',
'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING',
'SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
'SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE',
'SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING',
'SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD',
'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD',
'SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA',
'SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA',
'SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
'SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
'SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
'SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'

                      ), 
                      '6E70BFD2F17848BBB19A31ECDFB49EE7')  
                
 
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                          
          
                
                
     

      --   group by 1,2,3
      --   order by 4 desc 

         
""")

In [31]:
labResultsDf.limit(10).toPandas()

,tenant,personId,encounterId,labid,serviceDate,labcode,labcode_id,labcode_display,labcode_system_id,type,labcode_value,labcode_unitofmeasure_id,labcode_unitofmeasure_display,labcode_unitofmeasure_system_id,labcode_interpretation_id,labcode_interpretation_display,labcode_interpretation_system_id
0,63,6048bdf0-4040-419e-92af-d74d72f0b175,5dcc9af4-635c-41d9-834c-9a80d834731a,660a37b9-e1e0-4d53-baa8-813918d0be27,2021-03-10T17:05:00+00:00,"((94500-6, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection), [(94500-6, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection), (94309-2, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection)])",94500-6,SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection,2.16.840.1.113883.6.1,TEXT,Not Detected,None,None,None,385432009,Not applicable,2.16.840.1.113883.6.96
1,63,6048bdf0-4040-419e-92af-d74d72f0b175,5dcc9af4-635c-41d9-834c-9a80d834731a,d2633d66-002e-4925-8d47-9a004b1b8301,2021-03-05T11:37:00+00:00,"((94500-6, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection), [(94500-6, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection), (94309-2, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection)])",94500-6,SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection,2.16.840.1.113883.6.1,TEXT,Not Detected,None,None,None,385432009,Not applicable,2.16.840.1.113883.6.96
2,63,6048bdf0-4040-419e-92af-d74d72f0b175,06dc3f4b-32e4-48fd-8c61-8d26a5025999,fdb6cb7f-cb00-40ec-861c-ca9985153c39,2021-01-27T01:52:00+00:00,"((94500-6, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection), [(94500-6, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection), (94309-2, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection)])",94500-6,SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection,2.16.840.1.113883.6.1,TEXT,Detected,None,None,None,A,Abnormal,2.16.840.1.113883.12.78
3,63,6048bdf0-4040-419e-92af-d74d72f0b175,0293337b-653b-48ca-8db5-8acf7df8f267,303b51b0-a15d-41ce-9c89-9088bec69e04,2021-02-19T20:48:00+00:00,"((94500-6, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection), [(94500-6, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection), (94309-2, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection)])",94500-6,SARS-CoV-2 (COVID-19) RNA [Presence] in Respiratory specimen by NAA with probe detection,2.16.840.1.113883.6.1,TEXT,Detected,None,None,None,A,Abnormal,2.16.840.1.113883.12.78
4,63,853c06fb-1e5a-46c1-aa69-d6ec529d8d85,b904762b-1527-4308-9bfd-d4ada24e34d0,c6f8f5d8-72bb-4f80-a47e-78046617b6ae,2020-08-08T00:00:00+00:00,"((94309-2, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection), [(94309-2, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection)])",94309-2,SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection,2.16.840.1.113883.6.1,TEXT,Detected,None,None,None,A,Abnormal,2.16.840.1.113883.12.78
5,63,853c06fb-1e5a-46c1-aa69-d6ec529d8d85,d6d19eff-b8a7-434e-8ebb-9a72a8e94f4e,11213cc1-ff88-436b-8d33-31a7e1fec5b0,2020-07-29T22:45:00+00:00,"((94309-2, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection), [(94309-2, 2.16.840.1.113883.6.1, SARS-CoV-2 (COVID-19) RNA [Presence] in Specimen by NAA with probe detection)]

In [32]:
# Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_DetailedPatientsCoronavirusLabs""").show() 

++
||
++
++



In [33]:
labResultsDf.write.saveAsTable(f"""{userName}.{algorithmName}_DetailedPatientsCoronavirusLabs """)

In [11]:
labResultsTransposeDf = spark.sql(f"""
 select distinct l.tenant 
                ,l.personId
                ,l.encounterId
                ,l.serviceDate
                
          ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('COVID_19_EVALUATION_NOTE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_COVID_19_EVALUATION_NOTE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('COVID_19_INITIAL_EVALUATION_FORM'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_COVID_19_INITIAL_EVALUATION_FORM_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode.standard.id,null)) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_id



            ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('COVID_19_EVALUATION_NOTE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_COVID_19_EVALUATION_NOTE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('COVID_19_INITIAL_EVALUATION_FORM'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_COVID_19_INITIAL_EVALUATION_FORM_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), 1,0)) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_test
      
            ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('COVID_19_EVALUATION_NOTE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_COVID_19_EVALUATION_NOTE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('COVID_19_INITIAL_EVALUATION_FORM'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_COVID_19_INITIAL_EVALUATION_FORM_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_value,null)) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_value

          
            ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_ARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('COVID_19_EVALUATION_NOTE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_COVID_19_EVALUATION_NOTE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('COVID_19_INITIAL_EVALUATION_FORM'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_COVID_19_INITIAL_EVALUATION_FORM_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
      ,MAX(IF(HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, array('SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'),'6E70BFD2F17848BBB19A31ECDFB49EE7'), labcode_interpretation_display,null)) as taken_SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION_lab_interpretation
    
          
   from {userName}.{algorithmName}_DetailedPatientsCoronavirusLabs l
   
   
   
   group by 1,2,3,4 
   order by 1,2,3,4
   
""")

In [16]:
labResultsTransposeDf.limit(150).toPandas().to_excel("test2.xlsx")

In [13]:
# Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_SummaryPatientsCoronavirusLabs""").show() 

++
||
++
++



In [14]:
labResultsTransposeDf.write.saveAsTable(f"""{userName}.{algorithmName}_SummaryPatientsCoronavirusLabs """)

In [56]:
##spark.sql(f"""select * from {userName}.{algorithmName}_SummaryPatientsCoronavirusLabs""").limit(150).toPandas().to_excel("test.xlsx")

In [21]:
# labSummaryDf = spark.sql(f"""
#    select h.personId
#          ,h.encounterId
#          ,l.labid
#          ,l.serviceDate
#          ,l.labcode.standard.id as labcode_id
#          ,l.labcode.standard.primaryDisplay as labcode_display
#          ,case  
#            when l.type = 'NUMERIC' then cast(typedvalue.numericValue.value as string)
#            when l.type = 'TEXT' then cast(typedvalue.textvalue.value as string)
#            when l.type = 'DATE' then cast(typedvalue.datevalue.date as string)
#            else cast(typedvalue.codifiedValues.values.value.standard.primaryDisplay as string)  
#           end labcode_value   
#          ,l.unitofmeasure.standard.primaryDisplay as labcode_unitofmeasure_display
#          ,l.interpretation.standard.primaryDisplay as labcode_interpretation_display
   

  
#         FROM {userName}.{algorithmName}_Hospital h  
        

       

#         JOIN {db}.lab l
#             ON  h.encounterId = l.encounterId
#                 and h.personId = l.personID
#                 and h.tenant = l.tenant
                
#                  AND HAS_ANY_CONCEPT_IN_CONTEXT(l.labcode, 
#                     array('COVID_19_EVALUATION_NOTE',
# 'COVID_19_INITIAL_EVALUATION_FORM',
# 'EMERGENCY_DEPARTMENT_COVID_19_INITIAL_EVALUATION_FORM',
# 'INFLUENZA_A_B_RNA_AND_SARS_CORONAVIRUS_2_GENE_PANEL_RESPIRATORY_NAA_WITH_PROBE_DETECTION',
# 'INFLUENZA_A_B_SARS_COV_2_COVID_19_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPEC_NAA_PROBE_DETECTION',
# 'INFLUENZA_VIRUS_A_AND_B_AND_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESPIRATORY_BY_RAPID_IMMUNOASSAY',
# 'INFLUENZA_VIRUS_A_B_AND_SARS_CORONAVIRUS_2_IDENTIFIED_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION',
# 'INFLUENZA_VIRUS_A_B_AND_SARS_COV_2_COVID_19_AND_RSV_RNA_PANEL_RESP_SPECIMEN_BY_NAA_PROBE_DETECTION',
# 'INFLUENZA_VIRUS_A_B_AND_SARS_COV_SARS_COV_2_COVID_19_AG_PANEL_UPPER_RESP_SPEC_BY_RAPID_IMMUNOASSAY',
# 'MIDDLE_EAST_RESPIRATORY_SYNDROME_CORONAVIRUS_RNA_PRESENCE_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
# 'PERCENT_NEUTRALIZATION_BY_SARS_COV_2_SPIKE_PROTEIN_RBD_NEUTRALIZING_AB_IN_SERUM_OR_PLASMA_BY_IA',
# 'POLYMERASE_CHAIN_REACTION_PCR_TEST_FOR_2019_NOVEL_CORONAVIRUS',
# 'RESPIRATORY_VIRAL_PATHOGENS_DNA_AND_RNA_PANEL_RESPIRATORY_SPECIMEN_QUALITATIVE_BY_NAA_WITH_PROBE',
# 'SARA_COV_2_COVID_19_IGG_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
# 'SARS_C2_C13_COV_2_COVID_19_ORFLAB_REGION_CYCLE_THRESHOLD_OROPHARYNGEAL_WASH_NAA_PROBE_DETECTION',
# 'SARS_CORONAVIRUS_2_N_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_NAA_WITH_PROBE_DETECTION',
# 'SARS_CORONAVIRUS_2_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING',
# 'SARS_COV_2_COVID19_AB_INTERPRETATION_IN_SERUM_OR_PLASMA',
# 'SARS_COV_2_COVID19_AB_PANEL_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_E_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_E_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGA_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGA_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGA_IGM_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGG_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGG_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGG_AND_IGM_PANEL_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGG_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASSASY',
# 'SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_OR_PLASMA_BY_IMMUNOASAY',
# 'SARS_COV_2_COVID19_IGM_AB_PRESENCE_IN_SERUM_PLASMA_OR_BLOOD_BY_RAPID_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_IGM_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SERUM_BY_PVNT',
# 'SARS_COV_2_COVID19_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_BY_PVNT',
# 'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
# 'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
# 'SARS_COV_2_COVID19_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N1',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_CDC_PROBE_SET_N2',
# 'SARS_COV_2_COVID19_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
# 'SARS_COV_2_COVID19_ORF1AB_REGION_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
# 'SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_ORF1AB_REGION_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_ORGANISM_SPECIFIC_CULTURE',
# 'SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
# 'SARS_COV_2_COVID19_RDRP_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
# 'SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RDRP_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_LOG_VOLUME_VIRAL_LOAD_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE',
# 'SARS_COV_2_COVID19_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_PANEL_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_NON_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NASOPHARYNX_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_PRESENCE_IN_NOSE_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_SEQUENCING',
# 'SARS_COV_2_COVID19_RNA_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_S_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_SERUM_OR_PLASMA_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_S_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID19_WHOLE_GENOME_NUCLEOTIDE_SEQUENCE_IN_ISOLATE_BY_SEQUENCING',
# 'SARS_COV_2_COVID_19_AB_PANEL_DBS_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_AB_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_RAPID_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_AG_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_AND_SARS_RELATED_COV_RNA_PANEL_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECT',
# 'SARS_COV_2_COVID_19_CLADE_TYPE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
# 'SARS_COV_2_COVID_19_E_GENE_CYCLE_THRESHOLD_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_E_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_IGA_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
# 'SARS_COV_2_COVID_19_IGG_AB_MASS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_IGG_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
# 'SARS_COV_2_COVID_19_IGM_AB_PRESENCE_IN_DBS_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_IGM_AB_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOFLUORESCENCE',
# 'SARS_COV_2_COVID_19_LINEAGE_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
# 'SARS_COV_2_COVID_19_M_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_NSP2_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_N_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_N_GENE_LOG_VOLUME_VIRAL_LOAD_RESPIRATORY_SPECIMEN_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_NOSE_BY_NAA_WITH_NON_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_BY_NAA_USING_CDC_PRIMER_PROBE_SET_N1',
# 'SARS_COV_2_COVID_19_N_GENE_PRESENCE_IN_SALIVA_ORAL_FL_BY_NUC_ACID_AMP_USING_CDC_PRIMER_PROBE_SET_N2',
# 'SARS_COV_2_COVID_19_N_GENE_VOLUME_VIRAL_LOAD_RESPIRATORY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_ORF1AB_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_ORF1AB_REGION_UNITS_VIRAL_LOAD_UPPER_RESPIRATORY_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_ORF1A_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_ORF1B_REGION_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_RDRP_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_RNA_PANEL_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_RNA_PANEL_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_NON_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_RNA_PRESENCE_IN_SPECIMEN_FROM_DONOR_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_SPECIFIC_TCRB_GENE_REARRANGEMENTS_PRESENCE_IN_BLOOD_BY_SEQUENCING',
# 'SARS_COV_2_COVID_19_S_GENE_CODON_N501Y_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
# 'SARS_COV_2_COVID_19_S_GENE_CODON_N501_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
# 'SARS_COV_2_COVID_19_S_GENE_CYCLE_THRESHOLD_IN_OROPHARYNGEAL_WASH_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_S_GENE_MUTATION_DETECTED_IDENTIFIER_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
# 'SARS_COV_2_COVID_19_S_GENE_MUTATION_PRESENCE_IN_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
# 'SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_SEQUENCING',
# 'SARS_COV_2_COVID_19_S_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_AB_UNITS_VOLUME_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_PRESENCE_IN_SER_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_S_PROTEIN_RBD_NEUTRALIZING_ANTIBODY_TITER_IN_SERUM_OR_PLASMA_BY_IMMUNOASSAY',
# 'SARS_COV_2_COVID_19_VARIANT_INTERPRETATION_IN_SPECIMEN_NARRATIVE',
# 'SARS_COV_2_COVID_19_VARIANT_TYPE_IN_SPECIMEN_BY_SEQUENCING',
# 'SARS_COV_2_COVID_19_WHOLE_GENOME_SEQUENCING_AND_ID_PANEL_SPECIMEN_BY_MOLECULAR_GENETICS_METHOD',
# 'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PANEL_BLOOD',
# 'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_PRESENCE_IN_BLOOD',
# 'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_CORRECTED_BACKGROUND_IN_BLD',
# 'SARS_COV_2_STIMULATED_GAMMA_INTERFERON_RELEASE_BY_T_CELLS_UNITS_VOLUME_IN_BLOOD',
# 'SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_HELPER_CD4_T_CELLS_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA',
# 'SARS_COV_2_STIM_GAM_INTERFERON_REL_BY_LYMPHOCYTES_UNITS_VOLUME_CORR_FOR_BKGRND_IN_BLD_BY_IA',
# 'SARS_COV_SARS_COV_2_COVID19_AG_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_RAPID_IMMUNOASSAY',
# 'SARS_LIKE_CORONAVIRUS_N_GENE_CYCLE_THRESHOLD_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_LIKE_CORONAVIRUS_N_GENE_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_LOWER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_SALIVA_ORAL_FLUID_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_RELATED_CORONAVIRUS_E_GENE_PRESENCE_IN_UPPER_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_RELATED_CORONAVIRUS_MERS_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE',
# 'SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_RESPIRATORY_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION',
# 'SARS_RELATED_CORONAVIRUS_RNA_PRESENCE_IN_UNSPECIFIED_SPECIMEN_BY_NAA_WITH_PROBE_DETECTION'

#                       ), 
#                       '6E70BFD2F17848BBB19A31ECDFB49EE7')  
                
 
#                 AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
#                                                        'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
#                                                        'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                          
          
                
                
      
                               
#       --   group by 1,2,3
#       --   order by 4 desc 

         
# """)

In [22]:
# # Drop existing table, if there was one 
# spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_SummaryPatientsCoronavirusLabs""").show() 

++
||
++
++



In [ ]:
# labSummaryDf.write.saveAsTable(f"""{userName}.{algorithmName}_SummaryPatientsCoronavirusLabs """)